In [ ]:
!pip install bert-for-tf2
!pip install tensorflow-text

     |████████████████████████████████| 51kB 5.0MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=12f9838dd5c0339c39a58fb31cbf1c1ecafca1b16f5ec2889fcdcc1e1c53fb0c
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7303 sha256=457b46d6878600bca0f0859603f8253d0ae2caba1af9f04a313c44e189c4a1e3
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=70fec8ca79eb27fe8e34fcb5b9f81b5926eaab313812bd659664856b3e911cfd
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 3.4MB 7.9MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_model(model_url, max_seq_length):
  inputs = dict(
    input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    )

  muril_layer = hub.KerasLayer(model_url, trainable=True)
  outputs = muril_layer(inputs)

  assert 'sequence_output' in outputs
  assert 'pooled_output' in outputs
  assert 'encoder_outputs' in outputs
  assert 'default' in outputs
  return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer

In [ ]:
max_seq_length = 128
muril_model, muril_layer = get_model(
    model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=max_seq_length)

In [ ]:
vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def create_input(input_strings, tokenizer, max_seq_length):
  input_ids_all, input_mask_all, input_type_ids_all = [], [], []
  for input_string in input_strings:
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)
    
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    input_type_ids_all.append([0] * max_seq_length)
  
  return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [ ]:
def encode(input_text):
  input_ids, input_mask, input_type_ids = create_input(input_text, 
                                                       tokenizer, 
                                                       max_seq_length)
  inputs = dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
  )
  return muril_model(inputs)

In [ ]:
code_mix_sentences = ["मे घर जाऊंगा","मैं घर जा रही हूँ","i am going home",'main ghar ja raha hoon','apka naam kya hai']

In [ ]:
code_mix_embedding = encode(code_mix_sentences)

In [ ]:
dst_1 = cosine_similarity(np.array(code_mix_embedding[0]).reshape(1,-1), 
                           np.array(code_mix_embedding[1]).reshape(1,-1))
print("Distance between {} & {} is {}".format(code_mix_sentences[0],
                                                code_mix_sentences[1],
                                                dst_1))

Distance between मे घर जाऊंगा & मैं घर जा रही हूँ is [[0.9994648]]


In [ ]:
dst_1 = cosine_similarity(np.array(code_mix_embedding[0]).reshape(1,-1), 
                           np.array(code_mix_embedding[2]).reshape(1,-1))
print("Distance between {} & {} is {}".format(code_mix_sentences[0],
                                                code_mix_sentences[2],
                                                dst_1))

Distance between मे घर जाऊंगा & i am going home is [[0.99958205]]


In [ ]:
dst_1 = cosine_similarity(np.array(code_mix_embedding[0]).reshape(1,-1), 
                           np.array(code_mix_embedding[4]).reshape(1,-1))
print("Distance between {} & {} is {}".format(code_mix_sentences[0],
                                                code_mix_sentences[4],
                                                dst_1))

Distance between मे घर जाऊंगा & apka naam kya hai is [[0.99903125]]


In [ ]:
dst_1 = cosine_similarity(np.array(code_mix_embedding[0]).reshape(1,-1), 
                           np.array(code_mix_embedding[3]).reshape(1,-1))
print("Distance between {} & {} is {}".format(code_mix_sentences[0],
                                                code_mix_sentences[3],
                                                dst_1))

Distance between मे घर जाऊंगा & main ghar ja raha hoon is [[0.9992978]]
